This program takes 2D image, assigns one GPU thread to each pixel, applies a threshold to every pixel in parallel, and produces a binary segmentation mask


In [22]:
%%writefile pixel_threshold_2d.cu
#include <stdio.h>

__global__ void threshold_kernel_2d(
    float* input,
    int* output,
    int width,
    int height,
    float thresh
) {
    int x = blockIdx.x * blockDim.x + threadIdx.x; // column
    int y = blockIdx.y * blockDim.y + threadIdx.y; // row

    if (x < width && y < height) {
        int idx = y * width + x;
        output[idx] = (input[idx] > thresh) ? 1 : 0;
    }
}

int main() {
    // Image dimensions
    int width = 4;
    int height = 4;
    int N = width * height;

    // Host (CPU) memory
    float h_input[N];
    int h_output[N];

    for (int i = 0; i < N; i++) {
        h_input[i] = i * 0.1f;
    }

    // Device (GPU) memory
    float* d_input;
    int* d_output;

    cudaMalloc(&d_input, N * sizeof(float));
    cudaMalloc(&d_output, N * sizeof(int));

    cudaMemcpy(d_input, h_input, N * sizeof(float), cudaMemcpyHostToDevice);

    // 2D execution configuration
    dim3 threadsPerBlock(16, 16);
    dim3 blocks(
        (width + threadsPerBlock.x - 1) / threadsPerBlock.x,
        (height + threadsPerBlock.y - 1) / threadsPerBlock.y
    );

    threshold_kernel_2d<<<blocks, threadsPerBlock>>>(
        d_input, d_output, width, height, 0.5f
    );
    cudaDeviceSynchronize();

    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA error: %s\n", cudaGetErrorString(err));
    }

    cudaMemcpy(h_output, d_output, N * sizeof(int), cudaMemcpyDeviceToHost);

    // Print output as 2D image
    for (int y = 0; y < height; y++) {
        for (int x = 0; x < width; x++) {
            printf("%d ", h_output[y * width + x]);
        }
        printf("\n");
    }

    cudaFree(d_input);
    cudaFree(d_output);
    return 0;
}


Writing pixel_threshold_2d.cu


In [25]:
!nvcc -arch=sm_75 pixel_threshold_2d.cu -o pixel_threshold_2d


In [26]:
!./pixel_threshold_2d


0 0 0 0 
0 0 1 1 
1 1 1 1 
1 1 1 1 
